We are going to use the python package keras to build a deep neural network to estimate the prices of the houses by training on our training set and testing on our test set.

First we'll import the necessary packages and load our data.

In [21]:
#To do: Regularization, try not setting parameters to 0/missing, try removing outliers, try removing unnecessary columns

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD,RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split

TrainingSet = pd.read_csv('data/train.csv')
TestSet = pd.read_csv('data/test.csv')
#Y = TrainingSet['SalePrice']

#Use logarithms to make the data set more closely match a normal distribution
Y = np.log1p(TrainingSet[['SalePrice']])
#Y = TrainingSet[['SalePrice']]
Y = Y.SalePrice

#We'll exclude the columns for X that we decided weren't worthwhile when visualizing the data
#As of now these are 'Utilities' (9),'PoolArea','MoSold' (76),'GarageArea' (62), GarageYrBlt' (59), 
# also 'ID' (0) and 'Price' (81)
X = TrainingSet[['MSSubClass','MSZoning','LotFrontage','LotArea','Street','Alley','LotShape',
 'LandContour', 'LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
 'OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st',
 'Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','Foundation','BsmtQual',
 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2',
 'BsmtUnfSF','TotalBsmtSF','Heating','HeatingQC','CentralAir','Electrical','1stFlrSF',
 '2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath',
 'BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu',
 'GarageType','GarageFinish','GarageCars','GarageQual','GarageCond',
 'PavedDrive','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
 'PoolQC','Fence','MiscFeature','MiscVal','YrSold','SaleType', 'SaleCondition']].copy()

XTest = TestSet[['MSSubClass','MSZoning','LotFrontage','LotArea','Street','Alley','LotShape',
 'LandContour', 'LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle',
 'OverallQual','OverallCond','YearBuilt','YearRemodAdd','RoofStyle','RoofMatl','Exterior1st',
 'Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond','Foundation','BsmtQual',
 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2',
 'BsmtUnfSF','TotalBsmtSF','Heating','HeatingQC','CentralAir','Electrical','1stFlrSF',
 '2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath',
 'BedroomAbvGr','KitchenAbvGr','KitchenQual','TotRmsAbvGrd','Functional','Fireplaces','FireplaceQu',
 'GarageType','GarageFinish','GarageCars','GarageQual','GarageCond',
 'PavedDrive','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
 'PoolQC','Fence','MiscFeature','MiscVal','YrSold','SaleType', 'SaleCondition']].copy()

#To simplify our data, we'll set missing categorical data to a new category, 'MISSING'
CategoryFeatures = X.select_dtypes(include=[np.object])
#Create a new category 'MISSING' for data that's not available
for i in CategoryFeatures:
    X[i] = X[i].astype('category')
    XTest[i] = XTest[i].astype('category')
    if X[i].isnull().any():
        X[i] = X[i].cat.add_categories(['MISSING'])
        X[i] = X[i].fillna('MISSING') 
    if XTest[i].isnull().any():
        XTest[i] = XTest[i].cat.add_categories(['MISSING'])
        XTest[i] = XTest[i].fillna('MISSING') 

#To further simplify, we'll also set missing numerical data to 0.
#This can be changed later if our model is not making accurate predictions
NumericalFeatures = X.select_dtypes(include=[np.number])
#Fill in our missing numerical values with '0'
for i in NumericalFeatures:
    X[i] = X[i].astype('float')
    XTest[i] = XTest[i].astype('float')
    if X[i].isnull().any():
        X[i] = X[i].fillna(0.0)
    if XTest[i].isnull().any():
        XTest[i] = XTest[i].fillna(0.0) 

#Keras works best when we have exclusively numerical data
#pd.get_dummies will replace each categorical value with 'dummy' numerical data 
#However, this increases our input dimensionalities based upon how many different unique values were in the category columns
X = pd.get_dummies(X)
XTest = pd.get_dummies(XTest)

XColumnList = list(X.columns.values)
XTestColumnList = list(XTest.columns.values)

for i in XColumnList:
    if i not in XTest.columns:
        X = X.drop(columns=[i])
        
for i in XTestColumnList:
    if i not in X.columns:
        XTest = XTest.drop(columns=[i])
        
#Unfortunately, the columns in X and XTest may no longer be identical.
#We should delete columns that are not present in BOTH dataframes


#print(X.head(15))
XTest.head(15)#See that null values are gone, as are the columns we didn't want

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0
5,60.0,75.0,10000.0,6.0,5.0,1993.0,1994.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
6,20.0,0.0,7980.0,6.0,7.0,1992.0,2007.0,0.0,935.0,0.0,...,0,0,0,1,0,0,0,0,1,0
7,60.0,63.0,8402.0,6.0,5.0,1998.0,1998.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
8,20.0,85.0,10176.0,7.0,5.0,1990.0,1990.0,0.0,637.0,0.0,...,0,0,0,1,0,0,0,0,1,0
9,20.0,70.0,8400.0,4.0,5.0,1970.0,1970.0,0.0,804.0,78.0,...,0,0,0,1,0,0,0,0,1,0


We have our training set X and Y established, as well as our test set X which will be used later for predictions.

In [25]:
#Establish our model
X_tr, X_val, y_tr, y_val = train_test_split(X, Y, test_size = 0.33)
model = Sequential()
BatchNormalization()
model.add(Dense(1028,input_dim=len(X.columns),activation='relu')) #Input dimensions equal to # of columns in dataframe X
BatchNormalization()
model.add(Dense(1028,activation='relu')) #Input dimensions automatically accounted for in later layers
BatchNormalization()
#Dropout(0.2)
model.add(Dense(100,activation='relu'))
BatchNormalization()
#Dropout(0.2)
model.add(Dense(50))
BatchNormalization()
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [26]:
model.fit(X_tr,y_tr,validation_data=(X_val,y_val),epochs=30,batch_size=100)

Train on 978 samples, validate on 482 samples
Epoch 1/30
978/978 [==============================] - 1s 1ms/step - loss: 32015070779.6810 - acc: 0.0000e+00 - val_loss: 13656873975.5021 - val_acc: 0.0000e+00
Epoch 2/30
978/978 [==============================] - 0s 463us/step - loss: 16197664013.0879 - acc: 0.0000e+00 - val_loss: 5737247511.3693 - val_acc: 0.0000e+00
Epoch 3/30
978/978 [==============================] - 0s 465us/step - loss: 10300676930.4867 - acc: 0.0000e+00 - val_loss: 4745683503.8008 - val_acc: 0.0000e+00
Epoch 4/30
978/978 [==============================] - 0s 485us/step - loss: 4941506969.3906 - acc: 0.0000e+00 - val_loss: 3837678364.6805 - val_acc: 0.0000e+00
Epoch 5/30
978/978 [==============================] - 0s 474us/step - loss: 3991665039.9673 - acc: 0.0000e+00 - val_loss: 3778494097.5270 - val_acc: 0.0000e+00
Epoch 6/30
978/978 [==============================] - 0s 482us/step - loss: 3438973612.2372 - acc: 0.0000e+00 - val_loss: 3643131235.8506 - val_acc: 0.0

Now we have our model created, we want to make predictions

In [27]:
np.sqrt(model.evaluate(X_val,y_val))
preds = model.predict(np.array(XTest))
subm = pd.read_csv("data/sample_submission.csv")
subm.iloc[:,1] = np.array(model.predict(np.array(XTest)))
subm['SalePrice'] = np.expm1(subm[['SalePrice']]) #undo the logarithm transformation we did on y earlier
subm.to_csv('KerasSub.csv', index=None)

482/482 [==============================] - 0s 159us/step
